In [ ]:
! pip install langid
! pip install normalise
! pip install spacy
!pip install nltk


In [ ]:
! pip install keras


In [ ]:
!pip install --no-cache-dir tensorflow

In [ ]:
! pip install tensorflow --user

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from tqdm import tqdm
import multiprocessing as mp
import spacy 
import en_core_web_sm
#import matplotlib.pyplot as plt
import nltk
#nltk.download('universal_tagset')
#nltk.download('names')
#nltk.download('brown')
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import stopwords
from normalise import normalise
import string
import io
import re
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import langid
import sys
from tensorflow import keras
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense
import tensorflow as tf
from keras.utils import to_categorical
# Neural network
#!{sys.executable} -m spacy download en
#from spacy_langdetect import LanguageDetector
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

/home/valentina/anaconda3/envs/ai/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/valentina/anaconda3/envs/ai/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
Using TensorFlow backend.


In [ ]:
#load the subset of yelp dataset cleaned
df2 = pd.read_csv('subset_review.csv', sep='\t' )
del df2["Unnamed: 0"]


In [ ]:
df2

Starting text pre-processing

In [ ]:
%%time

#tokenization, stop word removal and punctuation removal using spacy

nlp = spacy.load('en')
df2['cleaned_text'] = df2['text'].apply(lambda x: [t.lemma_.lower() for t in nlp.tokenizer(x) if ((not t.is_stop) and ( not t.is_punct) and (not t.is_currency) and (not t.is_digit) and (not t.is_space) and (t.is_alpha)) ])


In [ ]:
#df2.to_csv('subset_review_spacy.csv', sep='\t')


In [2]:
df2 = pd.read_csv('subset_review_spacy.csv', sep='\t' )
del df2["Unnamed: 0"]
df2

,stars,text,language,cleaned_text
0,5,These guys are awesome. I have been coming her...,en,"['guy', 'awesome', 'come', 'year', 'restaurant..."
1,5,Stopped by on Tuesday to look for some Glock 2...,en,"['stopped', 'tuesday', 'look', 'glock', 'mags'..."
2,5,Had a great experience here. Staff was really ...,en,"['great', 'experience', 'staff', 'nice', 'atte..."
3,1,This place is a complete joke. My fiancé order...,en,"['place', 'complete', 'joke', 'fiancé', 'order..."
4,1,Can you put more salt on you fries really my l...,en,"['salt', 'fry', 'lip', 'burn', 'plz', 'okay', ..."
...,...,...,...,...
99590,2,Came on BlogTO's recommendation. This restaura...,en,"['came', 'blogto', 'recommendation', 'restaura..."
99591,4,Ordered delivery to my hotel. They beat their ...,en,"['ordered', 'delivery', 'hotel', 'beat', 'esti..."
99592,5,Deserves all five stars! Got all my questions ...,en,"['deserves', 'star', 'got', 'question', 'answe..."
99593,2,Let me cry for a second before I write this re...,en,"['let', 'cry', '2', 'write', 'review', 'cry', ..."


In [ ]:
"""
#sentence = "This restaurant is amazzzinggg! I'll come back for sure ewwwww"
sentence = "chillin'"

#import re
#s = re.sub(r'(.)\1+', r'\1', sentence) 
#print(s)
nlt = [t.lower() for t in normalise(word_tokenize(sentence)) if ( (not t in stopwords.words("english") ) & (t.isalpha())) ]
print("nltk: " ,nlt)
#spac = [t.lemma_.lower() for t in nlp.tokenizer(sentence) if ((not t.is_stop) & ( not t.is_punct) & (not t.is_currency) & (not t.is_digit) & (not t.is_space) & (t.is_alpha)) ]
#print("spacy: ", spac)
"""
df2

In [3]:
print(df2['cleaned_text'].str.len().max())
print(df2['cleaned_text'].str.len().mean())
#len(df2.loc[df2['text'].str.len()==4518, 'text'].values[0])
df2.loc[df2['cleaned_text'].str.len()==4518].values

4518
447.4214267784527


array([[5,
        'Full Review in Blog.\n\nA BYOB located in a rather blue collar part of town the restaurant may seem an odd choice to the unfamiliar-everything about its location, size, and layout seems to say "locals only." What those paying attention would realize, however, is that Chef Douglas Dick has twice been a Beard Award Semi-Finalist, thrice been named Pittsburgh Magazine\'s Chef of the Year, and has been embracing the "Farm to Table" movement since before most had American\'s even heard of it.  Locally sourced foods prepared by an internationally educated chef and a menu that changes daily (because Dick goes to the market daily for his products) everything about Bona Terra sounded fantastic.\n\nGiven our large lunch and long drive ahead it was decided that we would share a couple of small appetizer plates prior to our mains - and given the fact that neither appetizer required cooking they would arrive quite quickly after our amuse.  Starting first with a cheese plate, Cyp

In [4]:
df2['sentiment'] = df2['stars'].apply(lambda x: 0 if x < 3 else 1)
df2


,stars,text,language,cleaned_text,sentiment
0,5,These guys are awesome. I have been coming her...,en,"['guy', 'awesome', 'come', 'year', 'restaurant...",1
1,5,Stopped by on Tuesday to look for some Glock 2...,en,"['stopped', 'tuesday', 'look', 'glock', 'mags'...",1
2,5,Had a great experience here. Staff was really ...,en,"['great', 'experience', 'staff', 'nice', 'atte...",1
3,1,This place is a complete joke. My fiancé order...,en,"['place', 'complete', 'joke', 'fiancé', 'order...",0
4,1,Can you put more salt on you fries really my l...,en,"['salt', 'fry', 'lip', 'burn', 'plz', 'okay', ...",0
...,...,...,...,...,...
99590,2,Came on BlogTO's recommendation. This restaura...,en,"['came', 'blogto', 'recommendation', 'restaura...",0
99591,4,Ordered delivery to my hotel. They beat their ...,en,"['ordered', 'delivery', 'hotel', 'beat', 'esti...",1
99592,5,Deserves all five stars! Got all my questions ...,en,"['deserves', 'star', 'got', 'question', 'answe...",1
99593,2,Let me cry for a second before I write this re...,en,"['let', 'cry', '2', 'write', 'review', 'cry', ...",0


Bag of words 


In [ ]:
#todo: normalize words
df2.shape
positive = df2.head(1000).loc[(df2["sentiment"] == 1)  ]
negative = df2.head(1000).loc[(df2["sentiment"] == 0)  ]
print(positive.shape)
print(negative.shape)

In [5]:
#df2['cleaned_text'] = df2['cleaned_text'].astype(str)
positive_review = ["The restaurant was great. The food was delicious and the staff was extremely kind."]
negative_review = ["The restaurant was awful. The food was cold."]

vectorizer = CountVectorizer(max_features=2500)
#bow = df2['cleaned_text'].apply(lambda x: vectorizer.fit_transform(x))
#bow =  vectorizer.fit_transform(df2['cleaned_text'].head(20000))
vectorizer.fit(df2['cleaned_text'].head(40000))
# summarize
print(vectorizer.vocabulary_)
# encode document
X = vectorizer.transform(df2['cleaned_text'].head(40000)).toarray()
XX = vectorizer.transform(sentence).toarray()
# summarize encoded vector
print(X.shape)
print(type(X))
print(X)
#to do: find the right size for the vocabulary and the number of reviews to keep

{'guy': 974, 'awesome': 131, 'come': 429, 'year': 2482, 'restaurant': 1825, 'kitchen': 1170, 'wet': 2439, 'bar': 154, 'product': 1686, 'tv': 2316, 'help': 1013, 'bbq': 170, 'knife': 1171, 'set': 1934, 'like': 1228, 'business': 286, 'level': 1222, 'customer': 548, 'service': 1931, 'great': 952, 'send': 1925, 'client': 402, 'hear': 1004, 'wonderful': 2463, 'review': 1830, 'stopped': 2105, 'tuesday': 2311, 'look': 1251, 'soon': 2022, 'walk': 2402, 'welcome': 2435, 'ask': 102, 'need': 1417, 'minute': 1370, 'extremely': 764, 'helpful': 1014, 'definitely': 580, 'return': 1829, 'area': 88, 'experience': 756, 'staff': 2068, 'nice': 1426, 'attentive': 119, 'food': 852, 'delicious': 584, 'fresh': 876, 'recommend': 1767, 'place': 1600, 'complete': 448, 'joke': 1151, 'fiancé': 808, 'order': 1484, 'roll': 1851, 'happy': 992, 'hour': 1052, 'menu': 1340, 'regular': 1783, 'bring': 256, 'course': 505, 'take': 2174, 'sushi': 2156, 'average': 126, 'well': 2436, 'salt': 1878, 'fry': 888, 'burn': 283, 'oka

NameError: name 'sentence' is not defined

In [ ]:
print(vectorizer.get_feature_names()[1075])
#create machine learning model

In [7]:
y = df2.iloc[:, 4].head(40000).values
y.shape

(40000,)

In [8]:
print(X.shape)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.25, random_state=1)

print(X_test.shape)


(40000, 2500)
(10000, 2500)


In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


(30000, 2500)
(10000, 2500)
(10000,)


In [10]:
print(X_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_train.shape)
#ohe = OneHotEncoder()
y_train = to_categorical(y_train)
print(y_train.shape)


(22500, 2500)
(7500, 2500)
(7500,)
(10000, 2500)
(22500,)
(22500, 2)


In [ ]:
#model
classifier = RandomForestClassifier(n_estimators=30, random_state=0)
classifier.fit(X_train, y_train)



In [ ]:
y_pred = classifier.predict(XX)
print(y_pred)

#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
#print(accuracy_score(y_test, y_pred))

In [11]:

model = Sequential()
model.add(Dense(16, input_dim=2500, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                40016     
_________________________________________________________________
dense_2 (Dense)              (None, 12)                204       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 26        
Total params: 40,246
Trainable params: 40,246
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
history = model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=15, batch_size=64)

ValueError: Error when checking target: expected dense_3 to have shape (2,) but got array with shape (1,)

In [ ]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)